In [1]:
import pandas as pd
import numpy as np

from env import get_db_url

import wrangle as w

In [2]:
sql_query = '''SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet,
                taxvaluedollarcnt, yearbuilt, taxamount, fips
               FROM properties_2017'''

In [3]:
url = get_db_url('zillow')

In [4]:
df = w.get_zillow_data()

In [5]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


## Initial goals for cleaning
### * Bedroom count
    * change the column name
    * handle nulls - median?
    * convert to int  
    
### *  Bathroom count
    * change the column name
    * handle nulls
    
### * Square Footage
    * change column name
    * handle nulls
    * convert to int
    
### *Year Built
    * change column name?
    * handle nulls
    * convert from float to datetime, year only
    
### *Tax Amount
    * handle nulls

### *Fips
    * FIPS is the county identification code. All 3 counties in this data are in California.
    * handle nulls
    * drop decimal points and convert to object dtype.


In [7]:
df.bedroomcnt.value_counts()

3.0     964298
4.0     634289
2.0     335473
5.0     150866
6.0      25166
1.0      23166
0.0      13187
7.0       4807
8.0       1107
9.0        291
10.0       121
11.0        34
13.0        16
12.0        12
14.0         7
15.0         6
18.0         3
16.0         2
25.0         1
Name: bedroomcnt, dtype: int64

In [8]:
df.bathroomcnt.value_counts()

2.00     943589
3.00     422841
1.00     414324
2.50     142981
4.00      82155
1.50      31211
3.50      28518
5.00      28362
4.50      19506
0.00      13027
6.00      10747
5.50       6217
7.00       4394
8.00       1692
6.50       1333
9.00        713
7.50        384
10.00       325
11.00       146
8.50        110
12.00        73
9.50         50
13.00        39
14.00        25
15.00        17
0.50         16
10.50        14
16.00        12
18.00         8
20.00         6
17.00         4
1.75          3
12.50         3
11.50         3
19.50         1
14.50         1
32.00         1
19.00         1
Name: bathroomcnt, dtype: int64

In [9]:
# What are fips?
df.fips.value_counts()

6037.0    1431811
6059.0     555077
6111.0     165975
Name: fips, dtype: int64

In [10]:
# see how many nulls are in each column
df.isnull().sum() 

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [11]:
df.size

15070041

In [12]:
# compare how much data would be lost if nulls are simply dropped
df.dropna().size

14981645

### For nulls, for initial cleaning I am going to just drop the nulls. Doing so only accounts for a loss of approximately 2.5% of the data. 

In [24]:
df.head()

,bedcount,bathcount,sqfeet,value,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [27]:
# drop the nulls to do the calculation
df = df.dropna()

# testing to see if changing floats in 
(df.sqfeet.astype(int) != df.sqfeet).sum()

0

In [14]:
# testing acquire function
zillow = w.get_zillow_data()

In [15]:
zillow.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [16]:
# renaming the columns 
df = df.rename(columns={'bedroomcnt':'bedcount',
                        'bathroomcnt':'bathcount',
                        'calculatedfinishedsquarefeet': 'sqfeet',
                        'taxvaluedollarcnt': 'value',})

In [17]:
df.head()

,bedcount,bathcount,sqfeet,value,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [28]:
# trying to convert yearbuilt to datetime
pd.to_datetime(df['yearbuilt'].astype(int).astype(str),format='%Y')

4         2005-01-01
6         2011-01-01
7         1926-01-01
11        1972-01-01
14        1973-01-01
             ...    
2152856   2015-01-01
2152858   2015-01-01
2152859   2014-01-01
2152861   2015-01-01
2152862   2014-01-01
Name: yearbuilt, Length: 2140235, dtype: datetime64[ns]

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   bedcount   float64
 1   bathcount  float64
 2   sqfeet     float64
 3   value      float64
 4   yearbuilt  float64
 5   taxamount  float64
 6   fips       float64
dtypes: float64(7)
memory usage: 130.6 MB


In [30]:
df.head()

,bedcount,bathcount,sqfeet,value,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [31]:
df['bedcount'] = df.bedcount.astype(int)

In [32]:
df['yearbuilt'] = df.yearbuilt.astype(int).astype(str)

In [33]:
df['sqfeet'] = df.sqfeet.astype(int)

In [36]:
df['fips'] = df.fips.astype(int).astype(str)

In [37]:
# check changes
df.head()

,bedcount,bathcount,sqfeet,value,yearbuilt,taxamount,fips
4,4,2.0,3633,296425.0,2005,6941.39,6037
6,3,4.0,1620,847770.0,2011,10244.94,6037
7,3,2.0,2077,646760.0,1926,7924.68,6037
11,0,0.0,1200,5328.0,1972,91.60,6037
14,0,0.0,171,6920.0,1973,255.17,6037


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   bedcount   int64  
 1   bathcount  float64
 2   sqfeet     int64  
 3   value      float64
 4   yearbuilt  object 
 5   taxamount  float64
 6   fips       object 
dtypes: float64(3), int64(2), object(2)
memory usage: 130.6+ MB


In [40]:
df.yearbuilt.describe()

count     2140235
unique        153
top          1955
freq        84329
Name: yearbuilt, dtype: object

## * when exploring the data, consider grouping the data by the discrete features to get different insights